# Setup

lsgraph was created with the aim of supporting reskilling and upskilling efforts for both individuals and organisations. lsgraph provides services to: 

* collect course details and associate with skills, 
* recommend courses for a learner, 
* recommend job profiles, 
* suggest reskilling options across a workforce.

This notebook walks through the process of setting up organisations, skills, profiles and users.

In [1]:
import json
import os
import pandas as pd
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000/api/v1/"

# Create credentials

Access to the API is restricted by API keys that need to be included in the request headers.

If running lsgraph locally via the docker containers the below command should create a new customer and access credentials.

In [3]:
!python ../create-customer-account.py "Customer Name" newcustomer@test.local

/home/jonathan/miniconda3/lib/python3.6/site-packages/apispec/ext/marshmallow/common.py:131: UserWarning: Multiple schemas resolved to the name User. The name has been modified. Either manually add each of the schemas with a different name or provide a custom schema_name_resolver.
  UserWarning,
/home/jonathan/miniconda3/lib/python3.6/site-packages/apispec/ext/marshmallow/common.py:131: UserWarning: Multiple schemas resolved to the name Skill. The name has been modified. Either manually add each of the schemas with a different name or provide a custom schema_name_resolver.
  UserWarning,
Customer ID: 51e37ad6-652b-471a-89ae-c5c1e8c8d300
API Key: LSTdYmT1QeDKNiIBCffK
Access Token: mbWJGt8i1iATUIE9fph3FQXWIoDIiRNBMvRaBz26


In [4]:
# Create a headers dictionary with the keys generated

# Set these to values returned above
headers = {"X-API-Key":"LSTdYmT1QeDKNiIBCffK", 
           "X-Auth-Token":"mbWJGt8i1iATUIE9fph3FQXWIoDIiRNBMvRaBz26"}

In [5]:
# Store credentials for use in other notebooks
with open("credentials.json", "w") as fp:
    json.dump(headers, fp)

# Create organisation

An organisation is a collection of users that share a skills graph and profiles.

In [6]:
# Create a new organisation

new_org = {"name":"Test organization",}

r = requests.post(base_url + "organizations/", json=new_org, headers=headers)
print("Status code", r.status_code)
print("Response", r.json())

organization_id = r.json()["id"]
root_skill_id = r.json()["root_skill"]["id"] # The root node for the skill graph used by this organization


Status code 200
Response {'id': '011509e6-e80f-487d-a5b8-8242b1078055', 'name': 'Test organization', 'root_skill': {'id': '1d7fa88c-e65c-45c7-94c0-d7295c3071ec'}}


In [7]:
# View all organisations

r = requests.get(base_url + "organizations/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'organizations': [{'id': '011509e6-e80f-487d-a5b8-8242b1078055', 'name': 'Test organization'}]}


In [8]:
# View more detail on one organisation

r = requests.get(base_url + f"organizations/{organization_id}/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'level_map': {'Advanced': 3.0, 'Beginner': 1.0, 'Expert': 4.0, 'Intermediate': 2.0, 'No knowledge': 0.0}, 'name': 'Test organization', 'root_skill': {'id': '1d7fa88c-e65c-45c7-94c0-d7295c3071ec'}}


The detail view also includes the skill level map. This can be customized when the organization is initially created.

# Create skills graph

Skills within an organization are structured into a graph.

A sample graph is included in the sample_data directory.

In [9]:
with open("sample_data/graph.json") as fp:
    graph = json.load(fp)

In [10]:
# The graph is stored as nested dictionaries

def add_skills(parent, subgraph):
    for k,v in subgraph.items():
        new_skill = {"name":k, "description":"", "parent":parent}
        r = requests.post(base_url + f"organizations/{organization_id}/skills/", 
                                 json=new_skill, 
                                 headers=headers)
        new_id = r.json()["id"]
        add_skills(new_id, v)

        
add_skills(root_skill_id, graph)

In [11]:
# View skill graph

r = requests.get(base_url + f"organizations/{organization_id}/skills/", headers=headers)
print("Status code", r.status_code)
print(len(r.json()["skills"]))
print("Response", r.json())

def build_full_path(skill):
    if skill["path"] == "Root":
        return skill["name"]
    return skill["path"][5:] + "|" + skill["name"]

all_skills = {build_full_path(i):i for i in r.json()["skills"]}

Status code 200
407
Response {'skills': [{'child_skills': [{'id': 'f2e5b1e5-352a-4472-ac52-7eb63e3164bd', 'name': 'personal skills'}, {'id': '7a5eac39-6942-4eaf-9646-9b871e70635f', 'name': 'art & design'}, {'id': '37f4d73f-8e12-49bc-a5bc-5fbb61b411c9', 'name': 'certifications'}, {'id': 'c72b0063-b2b7-46d6-8e0a-189f0b5ed3f1', 'name': 'data science'}, {'id': 'f9c34dbb-fd2b-45fa-a763-ed1086f22743', 'name': 'software engineering'}, {'id': '9b27ebf9-d9af-49e5-bdbf-202e57e56c78', 'name': 'information technology'}, {'id': 'e454bc94-c131-4d5b-a59d-5e4b12338514', 'name': 'engineering'}, {'id': '27699918-313d-435a-a9e8-23058c6c3d58', 'name': 'natural sciences'}, {'id': 'cc183181-49fd-4d47-9ddf-b233e824d3d9', 'name': 'languages'}, {'id': '8c556db3-53c2-4a92-b754-36e6190d5dbf', 'name': 'business & management'}, {'id': 'eef9ac56-ce77-42ac-a952-9719f4abce50', 'name': 'social sciences'}, {'id': '56e45647-2a5e-435b-9a84-a6971b1c1c09', 'name': 'mathematics'}, {'id': 'f0d076cd-8c56-44a4-9063-809364bb2ad

In [12]:
len(all_skills)

407

# Create profiles

Profiles are used to collect skills and associate levels.

A variety of sample profiles for specific jobs are available

In [13]:
# Load sample data

profile_dir = 'sample_data/profiles/'

level_map = {'B':'Beginner','I':'Intermediate','A':'Advanced','E':'Expert'}

def skill_id_map(graph, path):
    if len(path) == 1:
        return graph[path[0]]['_meta']['id']
    return skill_id_map(graph[path[0]], path[1:])

profiles = {}

for fn in os.listdir(profile_dir):
    profile_name = fn[:-len('.skills.csv')]
    data = pd.read_csv(os.path.join(profile_dir, fn), header=None)
    profile_competences = {}
    for _,competence in data.iterrows():
        level = competence[0]
        path="|".join([i for i in competence[1:].tolist() if isinstance(i, str)])
        profile_competences[all_skills[path]["id"]] = level_map[level]
    profiles[profile_name] = profile_competences

In [14]:
profiles

{'Applications Engineer': {'424bb798-c72c-4990-bbfe-65401e48a294': 'Intermediate',
  '567902a2-90ad-4c56-bd29-c480524afdf7': 'Advanced',
  '5a12fb3f-6879-458f-98f4-97f25132d6a3': 'Advanced',
  '629e46f8-12b0-4655-afc9-d6b475b55d5c': 'Advanced',
  '804912aa-5505-4da9-9a50-4eb04f285df3': 'Intermediate',
  'bb4d03b4-9f4c-4252-95ac-f3f3579ee739': 'Advanced',
  'c53d24fe-7c2f-4228-bc17-2632be79c601': 'Expert'},
 'Business Intelligence Analyst': {'8f2b39e0-c787-469e-8dc1-3867f2f57865': 'Advanced',
  'a7097563-6c5b-45d2-97cf-639b94849d7d': 'Expert',
  'b6920efa-4b19-46eb-908c-ea053bc0e3fc': 'Advanced',
  'c3d7b422-836f-4936-bc01-91737cec39bf': 'Advanced',
  'da1c7012-0167-4fee-be12-23a242f03361': 'Advanced'},
 'Chief Information Security Officer': {'1e3cb515-6644-4385-aafc-d352e4f90677': 'Expert',
  '1fc7fe80-f48b-4fc4-ada0-afdb781d8619': 'Advanced',
  '217451a8-ef74-45d6-964c-16c0188a100e': 'Expert',
  '27d1e909-ab00-4485-8029-87ae2ab0bbf7': 'Advanced',
  '43c3a53e-5a45-4b4c-a1a1-7e8e79a8767

In [15]:
# Load all profiles

for profile_name, competences in profiles.items():
    print("Creating profile", profile_name)
    new_profile = {"name":profile_name,
                   "description":"",
                   "type":"job role",
                   "skills":[{"skill":k, "level_name":v} for k,v in competences.items()]
                  }
    r = requests.post(base_url + f"organizations/{organization_id}/profiles/", json=new_profile, headers=headers)
    if r.status_code != 200:
        print("Status code", r.status_code)
        print("Error with", profile_name)
        break

Creating profile Applications Engineer
Creating profile Data Scientist
Creating profile Computer and Information Systems Managers
Creating profile Web Developers
Creating profile Software Engineer
Creating profile Software Engineer ML
Creating profile Systems Engineer
Creating profile DevOps Engineer
Creating profile Machine Learning Researcher
Creating profile Business Intelligence Analyst
Creating profile Software Developers, Applications
Creating profile Cloud architect
Creating profile Sales Engineer
Creating profile Database Administrators
Creating profile Security Engineer
Creating profile Computer Hardware Engineers
Creating profile Data Engineer
Creating profile Product Manager
Creating profile UX Designer
Creating profile Solutions Architect
Creating profile Mobile Developer
Creating profile Machine Learning Engineer
Creating profile Information Security Analysts
Creating profile Data Analyst
Creating profile Full stack developer
Creating profile Systems Administrator
Creating

In [16]:
# List all profiles for an organisation

r = requests.get(base_url + f"organizations/{organization_id}/profiles/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'profiles': [{'description': '', 'id': 'b117cd55-be55-4a6b-a331-7c2a87ae3f5b', 'name': 'Applications Engineer', 'previous_versions': [], 'skills': [{'level': 4.0, 'level_name': 'Expert', 'skill': 'c53d24fe-7c2f-4228-bc17-2632be79c601'}, {'level': 2.0, 'level_name': 'Intermediate', 'skill': '424bb798-c72c-4990-bbfe-65401e48a294'}, {'level': 2.0, 'level_name': 'Intermediate', 'skill': '804912aa-5505-4da9-9a50-4eb04f285df3'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': '567902a2-90ad-4c56-bd29-c480524afdf7'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': '5a12fb3f-6879-458f-98f4-97f25132d6a3'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': '629e46f8-12b0-4655-afc9-d6b475b55d5c'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': 'bb4d03b4-9f4c-4252-95ac-f3f3579ee739'}], 'type': 'job role', 'user_id': None}, {'description': '', 'id': '3763b638-327a-4d18-9b27-ed81c048d565', 'name': 'Data Scientist', 'previous_versions': [], 'skills': [{'level': 4.0, '

# Create users

In [17]:
# Create four users with the competences associated with a job profile

users = [("Adam", profiles["Data Engineer"]), 
         ("Beth", profiles["Electrical Engineer"]), 
         ("Chris", profiles["Mobile Developer"]), 
         ("Dani", profiles["Security Engineer"])]

for user,competences in users:
    print("Creating user", user)
    new_user = {"email":"{0}@test.local".format(user),
                "name":user,
               }
    r = requests.post(base_url + f"organizations/{organization_id}/users/", json=new_user, headers=headers)
    print("Status code", r.status_code)
    print("Response", r.json())
    profile_id = r.json()["profile"]
    
    print("Updating profile")
    update_profile = {"skills":[{"skill":k, "level_name":v} for k,v in competences.items()]}
    r = requests.post(base_url + f"organizations/{organization_id}/profiles/{profile_id}/skills/", 
                      json=update_profile, 
                      headers=headers)
    if r.status_code != 200:
        print("Status code", r.status_code)
        print("Error with", profile_name)
        break
    print(r.json())

Creating user Adam
Status code 200
Response {'email': 'Adam@test.local', 'id': '8bd5e6ae-70cd-4afc-9a60-47961ae7a819', 'name': 'Adam', 'profile': '190e60cb-958e-478e-90b5-1b57f908e16c'}
Updating profile
{'skills': [{'level': 3.0, 'level_name': 'Advanced', 'skill': 'c53d24fe-7c2f-4228-bc17-2632be79c601'}, {'level': 4.0, 'level_name': 'Expert', 'skill': 'da1c7012-0167-4fee-be12-23a242f03361'}, {'level': 3.0, 'level_name': 'Advanced', 'skill': 'a1cac15e-2b06-433a-a257-d6bd6e7fedf3'}, {'level': 4.0, 'level_name': 'Expert', 'skill': '9385de27-cd84-42a4-ab05-f8cb04f1042c'}, {'level': 4.0, 'level_name': 'Expert', 'skill': 'fffc7570-09aa-4391-8bfe-c871dc0a121c'}, {'level': 4.0, 'level_name': 'Expert', 'skill': '2d51e534-cb98-4a69-be40-2ec583b7d59b'}, {'level': 4.0, 'level_name': 'Expert', 'skill': 'f60d2ee0-1218-4aeb-8959-dffd1399e60e'}]}
Creating user Beth
Status code 200
Response {'email': 'Beth@test.local', 'id': 'ad0ce93b-fad6-4c01-8e5f-2c63d13152cd', 'name': 'Beth', 'profile': '0c15a884-2

In [18]:
# List all users for an organisation

r = requests.get(base_url + f"organizations/{organization_id}/users/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'users': [{'email': 'Adam@test.local', 'id': '8bd5e6ae-70cd-4afc-9a60-47961ae7a819', 'name': 'Adam', 'profile': '190e60cb-958e-478e-90b5-1b57f908e16c'}, {'email': 'Beth@test.local', 'id': 'ad0ce93b-fad6-4c01-8e5f-2c63d13152cd', 'name': 'Beth', 'profile': '0c15a884-2706-450d-a4c7-42bf9cf53531'}, {'email': 'Chris@test.local', 'id': 'ca1e6a6c-77b1-4ae6-9a06-b214cdc253e0', 'name': 'Chris', 'profile': '621d72d9-d721-4aee-90d6-66abef9bb46f'}, {'email': 'Dani@test.local', 'id': '89c5d4e8-8899-40e5-86f2-fc1f5898eec1', 'name': 'Dani', 'profile': '827a2192-c14c-4d92-b910-93e9e7f07d6a'}]}
